# bookcorpus 加载

In [ ]:
from datasets import concatenate_datasets, load_dataset, load_from_disk
bookcorpus = load_from_disk("/root/autodl-tmp/cache/bookcorpus")

# wikipedia 加载
- parquet文件
- wikipedia/data/<subname>

In [6]:
from datasets import concatenate_datasets, load_dataset, load_from_disk
wiki1 = load_dataset("parquet", data_dir="/root/autodl-tmp/cache/wikipedia/20220301.simple/")
wiki2 = load_dataset("parquet", data_dir="/root/autodl-tmp/cache/wikipedia/20220301.en/")
# 需要翻墙下载数据处理器“wikimedia/wikipedia”
# ds = load_dataset("wikimedia/wikipedia", data_files="/root/autodl-tmp/cache/wikipedia/20220301.simple/train-00000-of-00001.parquet")

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

KeyboardInterrupt: 

# SlimPajama-627B 加载
`load.py`文件有代码，应该能用

In [11]:
import datasets
from datasets import IterableDataset, DatasetDict
import numpy as np
from pathlib import Path

class IterableDatasetWithLength(IterableDataset):
    def __len__(self):
        return self.length
    
    def set_length(self, length: int):
        self.length = length
        return self

datasets.builder.IterableDataset = IterableDatasetWithLength

def get_length(split: str, block_size: int):
    tokens = {
        "train": 100*1024*1024*1024,
        "validation": 50*1024*1024,
        "test": 50*1024*1024,
    }
    return tokens[split] // block_size

def data_iterator(split: str, path: Path, block_size: int):
    counter = 0
    for file in sorted((path / split).rglob("*.npz")):
        print(file)
        data = np.load(file)['input_ids'].astype(np.int64)
        for i in range(len(data) // block_size):

            if counter >= get_length(split, block_size):
                return
            counter += 1

            input_ids = data[i*block_size:(i+1)*block_size]
            attention_mask = np.ones_like(input_ids)
            yield {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': input_ids}

def get_datasets(path: str, block_size: int):
    print(path)
    datasets = DatasetDict({
        'train': IterableDataset.from_generator(data_iterator, gen_kwargs={"split": "train", "path": Path(path), "block_size": block_size}).set_length(get_length("train", block_size)),
        'validation': IterableDataset.from_generator(data_iterator, gen_kwargs={"split": "validation", "path": Path(path), "block_size": block_size}).set_length(get_length("validation", block_size)),
        'test': IterableDataset.from_generator(data_iterator, gen_kwargs={"split": "test", "path": Path(path), "block_size": block_size}).set_length(get_length("test", block_size)),
    })

    return datasets


In [ ]:
from torch.utils.data import IterableDataset, DataLoader

path_to_slimpajama_627B_tokenized = Path(".").absolute()
block_size = 2048

dataset = get_datasets(path_to_slimpajama_627B_tokenized, block_size)

batch_size=16
num_worker=0
train_dataloader = DataLoader(dataset, shuffle=False, batch_size=batch_size, num_workers=num_worker)

In [1]:
from transformers import LlamaForCausalLM, AutoTokenizer
tokenizer = LlamaTokenizer()

TypeError: LlamaTokenizer.__init__() missing 1 required positional argument: 'vocab_file'

In [12]:
path_to_data = Path("/root/autodl-tmp/cache/SlimPajama-627B-Llama-tokenized").absolute()
block_size = 2048

dataset = get_datasets(path_to_data, block_size)

/root/autodl-tmp/cache/SlimPajama-627B-Llama-tokenized


In [10]:
dataset["train"]

IterableDataset({
    features: Unknown,
    n_shards: 1
})

In [ ]:
import argparse
import os
import shutil
import time
import warnings
import numpy as np

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data
import torch.utils.data.distributed
from torch.utils.data.distributed import DistributedSampler
from models import DeepLab
from dataset import Cityscaples

parser = argparse.ArgumentParser(description='DeepLab')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('--epochs', default=100, type=int, metavar='N',
                    help='number of total epochs to run')
parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('-b', '--batch-size', default=3, type=int,
                    metavar='N')
parser.add_argument('--local_rank', default=0, type=int, help='node rank for distributed training')

args = parser.parse_args()
torch.distributed.init_process_group(backend="nccl") # 初始化 print("Use GPU: {} for training".format(args.local_rank))

# create model
model = DeepLab()

torch.cuda.set_device(args.local_rank) # 当前显卡 model = model.cuda() # 模型放在显卡上
model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True) # 数据并行
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), args.lr,
        momentum=args.momentum, weight_decay=args.weight_decay)

train_dataset = dataset["train"]
train_sampler = DistributedSampler(train_dataset) # 分配数据
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=args.batch_size, shuffle=False, num_workers=args.workers, pin_memory=True, sampler=train_sampler)

In [20]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler 
from torch.utils.data.distributed import DistributedSampler
from transformers import default_data_collator

train_dataset, eval_dataset = dataset["train"], dataset["validation"]
# DataLoaders creation:
if args.local_rank == -1:
    train_sampler = RandomSampler(train_dataset)
    eval_sampler = SequentialSampler(eval_dataset)
else:
    train_sampler = DistributedSampler(train_dataset) 
    eval_sampler = DistributedSampler(eval_dataset)
train_dataloader = DataLoader(train_dataset,
                              collate_fn=default_data_collator,
                              sampler=train_sampler,
                              batch_size=1)
eval_dataloader = DataLoader(eval_dataset,
                             collate_fn=default_data_collator,
                             sampler=eval_sampler,
                             batch_size=1)

ValueError: DataLoader with IterableDataset: expected unspecified sampler option, but got sampler=<torch.utils.data.sampler.RandomSampler object at 0x7fd4ab197d70>